In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib
# 'Agg' backend is for writing to file, not for rendering in a window.
matplotlib.use('Agg')
from IPython import embed

import matplotlib.pyplot as plt

In [2]:
class Sequence(nn.Module):
    # super() in python 2 version: super(Sequence, self).__init__() 
    def __init__(self):
        super(Sequence, self).__init__()
        self.lstm1 = nn.LSTMCell(1, 51)
        self.lstm2 = nn.LSTMCell(51, 51)
        self.linear = nn.Linear(51, 1)

    def forward(self, input, future = 0):
        outputs = []
        h_t = torch.zeros(input.size(0), 51, dtype=torch.double)
        c_t = torch.zeros(input.size(0), 51, dtype=torch.double)
        h_t2 = torch.zeros(input.size(0), 51, dtype=torch.double)
        c_t2 = torch.zeros(input.size(0), 51, dtype=torch.double)

        for i, input_t in enumerate(input.chunk(input.size(1), dim=1)):
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        for i in range(future):# if we should predict the future
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        outputs = torch.stack(outputs, 1).squeeze(2)
        return outputs


In [4]:
# set random seed to 0
np.random.seed(0)
torch.manual_seed(0)

In [7]:
# load data and make training set
data = torch.load('traindata.pt')

In [16]:
data

(100, 1000)

In [17]:
type(data)

numpy.ndarray

In [18]:
input = torch.from_numpy(data[3:, :-1])
target = torch.from_numpy(data[3:, 1:])
test_input = torch.from_numpy(data[:3, :-1])
test_target = torch.from_numpy(data[:3, 1:])

In [20]:
target.shape

torch.Size([97, 999])

In [21]:
test_target.shape

torch.Size([3, 999])

In [22]:
input.shape

torch.Size([97, 999])

In [23]:
test_input.shape

torch.Size([3, 999])

In [25]:
# build the model
seq = Sequence()

In [26]:
seq

Sequence(
  (lstm1): LSTMCell(1, 51)
  (lstm2): LSTMCell(51, 51)
  (linear): Linear(in_features=51, out_features=1, bias=True)
)

In [28]:
seq.double()
# Casts all floating point parameters and buffers to double datatype.

Sequence(
  (lstm1): LSTMCell(1, 51)
  (lstm2): LSTMCell(51, 51)
  (linear): Linear(in_features=51, out_features=1, bias=True)
)

In [30]:
# Creates a criterion that measures the mean squared error (squared L2 norm) between each element in the input xx and target yy .
criterion = nn.MSELoss()